Author: Ahmed Sobhi

Creation_date: 16th June 2023

Objective: Data EDA.

## Importing Required libararies and packages

In [1]:
import warnings
warnings.filterwarnings(action='once')

import pandas as pd
import numpy as np
# Used for visulization
import matplotlib.pyplot as plt
import seaborn as sns

# For iteration visulization purpose
from tqdm import tqdm

## Loading Dataset

In [2]:
# Loading dataset
df = (
    pd.read_csv('../data/intermid/DS-task-data-cleaned.csv')
    .assign(
        priced_at= lambda x: pd.to_datetime(x.priced_at)
    )
)

# Display first 5 rows
df.head()

,id,make,model,model_year,kilometers,transmission_type,price,priced_at,mileage_category,extra_features_count
0,17786,Nissan,Juke,2008,200000.0,Automatic,115000,2022-02-11,200k+,0.0
1,9997,Nissan,Juke,2008,200000.0,Automatic,116000,2022-04-21,200k+,0.0
2,8738,Nissan,Juke,2009,19999.0,Automatic,239000,2022-06-01,0-50k,0.0
3,9477,Nissan,Juke,2010,115000.0,Automatic,195000,2022-02-07,100k-150k,0.0
4,12801,Nissan,Juke,2011,19279.0,Automatic,248000,2022-06-29,0-50k,8.0


In [3]:
# Display dataframe info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18879 entries, 0 to 18878
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   id                    18879 non-null  int64         
 1   make                  18879 non-null  object        
 2   model                 18879 non-null  object        
 3   model_year            18879 non-null  int64         
 4   kilometers            18879 non-null  float64       
 5   transmission_type     18879 non-null  object        
 6   price                 18879 non-null  int64         
 7   priced_at             18879 non-null  datetime64[ns]
 8   mileage_category      18879 non-null  object        
 9   extra_features_count  18879 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(3), object(4)
memory usage: 1.4+ MB


In [5]:
# display dataframe statics
df.describe().T

,count,mean,std,min,25%,50%,75%,max
id,18879.0,10042.914455,5799.309902,0.0,5031.5,10048.0,15051.5,20097.0
model_year,18879.0,2016.192012,4.807641,1918.0,2014.0,2017.0,2020.0,2024.0
kilometers,18879.0,94909.778007,60216.631759,0.0,42000.0,90000.0,139999.0,285000.0
price,18879.0,272939.668415,129286.632260,10000.0,180000.0,247000.0,336500.0,1384000.0
extra_features_count,18879.0,8.765030,9.011134,0.0,0.0,7.0,15.0,40.0


- Price trend
- How many records per model
- Most common features

Feature eng:
- Moving average of price by x month.
- Car age.
- 

## Feature Engineering

### Car Age
- Describe the Age of car at the time of pricing.

In [5]:
# Car age at time the price was placed
df['model_age'] = df['priced_at'].dt.year - df['model_year']

# Dispaly head of df
df.head()

,id,make,model,model_year,kilometers,transmission_type,price,priced_at,mileage_category,extra_features_count,model_age
0,17786,Nissan,Juke,2008,200000.0,Automatic,115000,2022-02-11,200k+,0.0,14
1,9997,Nissan,Juke,2008,200000.0,Automatic,116000,2022-04-21,200k+,0.0,14
2,8738,Nissan,Juke,2009,19999.0,Automatic,239000,2022-06-01,0-50k,0.0,13
3,9477,Nissan,Juke,2010,115000.0,Automatic,195000,2022-02-07,100k-150k,0.0,12
4,12801,Nissan,Juke,2011,19279.0,Automatic,248000,2022-06-29,0-50k,8.0,11


### Price Moving Average.

In [7]:
# Price Moving Average
window_size = 3

df['price_moving_avg'] = (
    df.groupby('model')['price']
    .apply(
        lambda x: x.shift().rolling(window=window_size, min_periods=1).mean()
        )
    .fillna(method='bfill')
    .astype(int)
)

# Display head of dataframe
df.head()

/tmp/ipykernel_123842/3046543938.py:5: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  df.groupby('model')['price']


,id,make,model,model_year,kilometers,transmission_type,price,priced_at,mileage_category,extra_features_count,model_age,price_moving_avg
0,17786,Nissan,Juke,2008,200000.0,Automatic,115000,2022-02-11,200k+,0.0,14,115000
1,9997,Nissan,Juke,2008,200000.0,Automatic,116000,2022-04-21,200k+,0.0,14,115000
2,8738,Nissan,Juke,2009,19999.0,Automatic,239000,2022-06-01,0-50k,0.0,13,115500
3,9477,Nissan,Juke,2010,115000.0,Automatic,195000,2022-02-07,100k-150k,0.0,12,156666
4,12801,Nissan,Juke,2011,19279.0,Automatic,248000,2022-06-29,0-50k,8.0,11,183333
